In [ ]:
# pip install ipython seaborn matplotlib torchvision tqdm pillow pyyaml requests pandas opencv-python torch protobuf psutil
# Git clone 진행
# 혹시 git이 설치 안 된 상태라면 git 설치를 우선 해줘야 한다. 자세한 가이드라인은 따로 참조할 것.
# 1회만 실행하면 된다
!git clone https://github.com/ultralytics/yolov5
%pip install -qr requriements.txt

In [1]:
# 필요한 module upload
import torch
import yaml
from matplotlib import pyplot as plt
import numpy as np
import cv2
# 이미지를 한 번에 긁어모으기 위한 glob 모듈
from glob import glob
# Image를 open
from PIL import Image

In [2]:
# plt.imshow()후 커널 dead 현상 제거
# 실행 시 계속 실행
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# cuda 연산
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# 내 gpu 환경
torch.cuda.get_device_name(0), device

('GeForce RTX 3060 Laptop GPU', device(type='cuda'))

In [5]:
# yolov5 model load
# 따로 구축하진 않고 있는 parameter 활용
# parameter 활용 시 customizing 하는 방법 생각
model = torch.hub.load("ultralytics/yolov5", "yolov5x")

Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-30 Python-3.9.13 torch-1.13.1+cu116 CUDA:0 (GeForce RTX 3060 Laptop GPU, 6144MiB)



  0%|          | 0.00/166M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [6]:
# data set download
# 내 현재의 작업 환경으로 다운로드가 된다.
#!curl -L "https://public.roboflow.com/ds/eL4QUdkpSR?key=0ikL5WLM1w" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
# 작업환경 change
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5"

C:\Users\user\Desktop\recognition_project\yolov5


In [7]:
pwd

'C:\\Users\\user\\Desktop\\recognition_project\\yolov5'

In [8]:
# train image 경로, test image 경로를 통해 전반적으로 list로 모이게 된다
train_img = glob("./new_data_sets/train/images/*.jpg")
val_img = glob("./new_data_sets/valid/images/*.jpg")

YAML: 데이터를 표현하기 위한 포맷 중 하나
- 사람이 쉽게 읽고 쓸 수 있는 텍스트 기반 format, 파이썬의 딕셔너리와 유사한 형태로 데이터 표현
- YAML 파일은 Key-value 쌍으로 구성
- Key는 콜론으로 value와 구분, indentation을 통해 데이터의 계층 구조를 나타낸다
- pytorch와 같은 deep learning library에서는 모델 구성을 정의하는 yaml 파일을 사용하기도 한다
- YOLOv5 -> 모델 구성을 정의하기 위해 YAML 파일 활용. 이 파일에는 모델의 LAYER, HYPERPARMETER 등을 포함하며, 모델을 학습하기 전에 수정 가능하다. 이를 통해 모델 구성 실험, 최적의 성능을 찾을 수 있다.
- 그리고 YOLOv5 모델 학습을 위해서는 학습 데이터 및 라벨 데이터 모두를 YAML 파일로 표현해야 한다.
- yaml 파일에는 학습 데이터의 경로, 이미지와 라벨 파일의 이름, 이미지 및 라벨 파일의 경로 및 형식, 데이터셋 분할 방법 등의 정보가 포함되어 있다.
- label 파일을 표현하는 yaml 파일은 일반적으로 coco 형식을 따르며, 각 라벨의 이름, 인덱스, RGB 색상 등의 정보가 포함 된다.

!python detect.py --source "경로": 이 코드의 의미
- --source option: object detection을 수행할 이미지나 비디오 파일 또는 directory 경로를 저장한다.
- "경로" 디렉토리 내의 이미지 파일을 대상으로 object detection을 수행한다.
- YOLOv5 모델을 사용하여 "경로" 디렉토리 내의 이미지 파일에서 object detection을 수행하고, 결과를 출력하는 스크립트를 실행

In [7]:
# rough YOLOv5 detect
!python detect.py --source "./data/images/" --cfg "./yolov5x.yaml" --weights ./models/yolov5x.pt

python: can't open file 'C:\Users\user\Desktop\recognition_project\detect.py': [Errno 2] No such file or directory


In [16]:
img_paths = glob("runs/detect/exp2/*")
print(img_paths)
img = cv2.imread(img_paths[1])
plt.figure(figsize = (300, 300))
plt.imshow(img)
plt.show()

['runs/detect/exp2\\bus.jpg', 'runs/detect/exp2\\tkwls.png', 'runs/detect/exp2\\zidane.jpg']


In [ ]:
# data.yaml file open -> 해당 data set의 정보를 파악한다.
with open("./new_data_sets/data.yaml", "r") as f:
    data = yaml.load(f, Loader = yaml.FullLoader)
data["train"] = "./"
data["test"] = ""
data["val"] = ""
with open("", "w") as f:
    yaml.dump(data, f)

### 확인 결과
- train: train/images
- validation: valid/images
- nc: number of class -> 2개
- names: ["mask", "no-mask"]
- 이들은 annotation set이다

모델 학습 부분
- -epochs: epoch 수
- -data + data.yaml 파일 경로 (데이터셋 정보가 적힌 yaml 파일이다)
- -weights: Pre-trained 모델 파일 경로(pt 형식 파일)
- -batch: batch_size
- -cfg: yolov5 architecture yaml 파일 경로

In [ ]:
!python train.py --data "./yolov5/new_data_sets/data.yaml" --epochs 30 --batch 16 --cfg "./models/yolov5s.yaml" --weights ./models/yolov5s.pt

In [ ]:
ls